<a href="https://colab.research.google.com/github/Sushrut0202/CIIMS/blob/main/AMR_data_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()



Saving Cleaned AMR data.xlsx to Cleaned AMR data (2).xlsx


In [ ]:
import pandas as pd

# Replace this with your uploaded file name
filename = list(uploaded.keys())[0]

df = pd.read_excel(filename, header=0)

# Clean column names by stripping whitespace
df.columns = df.columns.str.strip()

pathogen_cols = [
    "E. coli", "S. aureus", "K. pneumoniae",
    "S. pneumoniae", "A.baumannii",
    "P. aeruginosa", "Others"
]

# Convert to long format
df_long = df.melt(
    id_vars=["Sr.No", "Institute ID", "Type", "Resistance"],
    value_vars=pathogen_cols,
    var_name="Pathogen",
    value_name="Present"
)

# Keep only detected pathogens
df_long = df_long[df_long["Present"] == "Yes"]

df_long.head()

,Sr.No,Institute ID,Type,Resistance,Pathogen,Present
3,4,CIIMS/AMR/004,Human,3.0,E. coli,Yes
5,6,CIIMS/AMR/006,Human,5.0,E. coli,Yes
6,7,CIIMS/AMR/007,Human,0.0,E. coli,Yes
7,8,CIIMS/AMR/008,Human,5.0,E. coli,Yes
8,9,CIIMS/AMR/009,Human,0.0,E. coli,Yes


In [ ]:
df.columns.tolist()


['Sr.No',
 'Institute ID',
 'Type',
 'E. coli',
 ' S. aureus',
 'K. pneumoniae',
 'S. pneumoniae',
 'A.baumannii',
 'P. aeruginosa',
 'Others',
 'Resistance']

In [ ]:
# Colab-ready: upload, reshape, save, download
from google.colab import files
import pandas as pd

# 1) Upload file
print("Upload your Excel file (Cleaned AMR data.xlsx)...")
uploaded = files.upload()  # pick your file

# 2) Read the uploaded file (first uploaded file)
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# 3) Clean column names (strip leading/trailing whitespace)
df.columns = df.columns.str.strip()
print("Cleaned columns:", df.columns.tolist())

# 4) Define pathogen columns (use the names you provided; now trimmed)
pathogen_cols = [
    "E. coli",
    "S. aureus",
    "K. pneumoniae",
    "S. pneumoniae",
    "A.baumannii",
    "P. aeruginosa",
    "Others"
]

# If any expected pathogen columns are missing, show warning
missing = [c for c in pathogen_cols if c not in df.columns]
if missing:
    print("Warning — these pathogen columns were NOT found in your sheet:", missing)
    # attempt auto-detect: treat any column that contains 'Yes' values as pathogen
    auto_candidates = [col for col in df.columns if df[col].astype(str).str.contains("Yes", na=False).any()]
    print("Auto-detected columns that contain 'Yes' values (candidates):", auto_candidates)
    # you can uncomment the next line to replace pathogen_cols with auto-detected candidates:
    # pathogen_cols = auto_candidates

# 5) Melt to long format
id_vars = ["Sr.No", "Institute ID", "Type", "Resistance"]
# Make sure id_vars exist
id_missing = [c for c in id_vars if c not in df.columns]
if id_missing:
    raise ValueError(f"These id_vars are missing from your sheet: {id_missing}")

df_long = df.melt(
    id_vars=id_vars,
    value_vars=[c for c in pathogen_cols if c in df.columns],
    var_name="Pathogen",
    value_name="Present"
)

# 6) Normalize Present column and keep only detected pathogens
df_long["Present"] = df_long["Present"].astype(str).str.strip().str.lower()
truthy = {"yes", "y", "1", "true", "detected"}
df_long = df_long[df_long["Present"].isin(truthy)]

# 7) Optional: if you want Resistance repeated as numeric
df_long["Resistance"] = pd.to_numeric(df_long["Resistance"], errors="coerce")

# 8) Save long-format file
outname = "AMR_long_format.xlsx"
df_long.to_excel(outname, index=False)
print(f"Long-format file saved: {outname} (rows: {len(df_long)})")

# 9) Create a simple pivot: pathogen counts
pivot_counts = df_long.pivot_table(index="Pathogen", values="Institute ID", aggfunc="count").rename(columns={"Institute ID":"Count"}).sort_values("Count", ascending=False)
print("\nPathogen counts (top):")
print(pivot_counts)

# 10) Download the file
files.download(outname)


Upload your Excel file (Cleaned AMR data.xlsx)...


Saving Cleaned AMR data.xlsx to Cleaned AMR data (3).xlsx
Cleaned columns: ['Sr.No', 'Institute ID', 'Type', 'E. coli', 'S. aureus', 'K. pneumoniae', 'S. pneumoniae', 'A.baumannii', 'P. aeruginosa', 'Others', 'Resistance']
Long-format file saved: AMR_long_format.xlsx (rows: 191)

Pathogen counts (top):
               Count
Pathogen            
E. coli           56
S. aureus         44
Others            31
K. pneumoniae     28
P. aeruginosa     24
A.baumannii        8


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>